In [ ]:
!conda install opencv
# %pip install opencv

In [1]:
!pip install pyzbar
%pip install ultralytics
%pip install zxing
%pip install cv2

/Users/henrikvendelbo/.zshenv:.:1: no such file or directory: /Users/henrikvendelbo/.cargo/env
/Users/henrikvendelbo/.zshenv:.:1: no such file or directory: /Users/henrikvendelbo/.cargo/env
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.7/898.7 kB 16.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 21.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
/Users/henrikvendelbo/.zshenv:.:1: no such file or directory: /Users/henrikvendelbo/.cargo/env
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 689.6/689.6 kB 1.9 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for zxing: filename=zxing-1.0.3-py3-none-any.whl size=684453 sha256=aaed2ed90f34ce93921a8676615290580691052485d1c2b62ffdde2327b6e21f
  Stored in directory: /Users/henrikvendelbo/Library/Caches/pip/wheels/99/34/5a/0dcc3878d8c58ac613fed5b14aa0e6b9794b28d5456e442493
Successfully built zxing
Note: you may need to restart the kernel t

## video

In [1]:

# Specify the path to the video file

import cv2
from zxing import BarCodeReader
import tempfile
import time
from ultralytics import YOLO
import numpy as np
def is_rectangle_contained(outer_rect, inner_rect):

    outer_x1, outer_y1, outer_x2, outer_y2 = outer_rect
    inner_x1, inner_y1, inner_x2, inner_y2 = inner_rect
    
    return inner_x1 >= outer_x1 and inner_y1 >= outer_y1 and inner_x2 <= outer_x2 and inner_y2 <= outer_y2

def print_rectangle_contained(results,results2):
    image = results[0].plot() & results2[0].plot()
    flag=0
    color1 = (0, 255, 0)  # Green color
    color2 = (255, 0, 0)
    thickness =2
    reader = BarCodeReader()
    for box in zip(results[0].boxes.xyxy,results[0].boxes.conf):
        flag=0
        for box2 in zip(results2[0].boxes.xywh,results2[0].boxes.conf):
            if(is_rectangle_contained(np.array(box[0]),np.array(box2[0]))):
                x1,y1,x2,y2 =np.array(box[0]) 
                rect1 = ((int(x1), int(y1)), (int(x2), int(y2)))
                cv2.rectangle(image, rect1[0],rect1[1], color1, thickness)
                x3,y3,width3,height3= np.array(box2[0])
                x,y,width,height = int(x3 - (width3)/2),int(y3 - (height3)/2),int(width3),int(height3)
                region_img = image[y:y+height, x:x+width]
                region_img_gray = cv2.cvtColor(region_img, cv2.COLOR_BGR2GRAY)
                # Save the cropped region as a temporary file
                with tempfile.NamedTemporaryFile(suffix='.png', delete=False) as temp_file:
                  #  print(temp_file.name)
                    temp_file_path = temp_file.name
                    cv2.imwrite(temp_file_path, region_img_gray)
               # gray = cv2.cvtColor(region_img, cv2.COLOR_BGR2GRAY)
               # buf = np.ascontiguousarray(gray).tobytes()
                
                barcode = reader.decode(temp_file_path)
                
                
                print('box :',np.array(box[0]),'conf:',box[1],'barcode :',np.array(box2[0]),'conf:',box2[1],'value : ',barcode.raw)
                flag=1
                
            
        if flag==0:
                x1,y1,x2,y2 =np.array(box[0]) 
                rect2 = ((int(x1), int(y1)), (int(x2), int(y2)))
                cv2.rectangle(image, rect2[0],rect2[1], color2, thickness)
                print('box :',np.array(box[0]),'does not contain barcode')
  
    cv2.imshow('image',image)
    
# Initialize the trained YOLO model
model_trained = YOLO('barcode_scanner.pt')

# Specify the path to the video file
video_path = 'video_2.mp4'

# Open the video file for reading
cap = cv2.VideoCapture(video_path)
frame_count = 0
# Check if the video opened successfully
if not cap.isOpened():
    print("Error: Unable to open video.")
else:
    # Loop to read and process each frame
    while True:
        # Read a frame from the video
        ret, frame = cap.read()
        frame_count += 1
        # Check if the frame was read successfully
        if not ret:
            print("End of video or unable to read frame.")
            break
        if frame_count % 20 != 0:
            continue   
       # frame_resized = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5) 
        image = frame #frame_resized #cv2.imread(frame)
        original_height, original_width, _ = image.shape
        
        model_trained = YOLO('best.pt')
        barcode_scanner_model = YOLO('barcode_scanner.pt')
        
        results = model_trained.predict(frame,imgsz=128)
        results2 = barcode_scanner_model.predict(frame,imgsz=(original_height, original_width))
        print_rectangle_contained(results,results2)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

FileNotFoundError: [Errno 2] No such file or directory: 'barcode_scanner.pt'

## open camera

In [14]:
import cv2
from zxing import BarCodeReader
import tempfile
import numpy as np
from ultralytics import YOLO

def is_rectangle_contained(outer_rect, inner_rect):
    outer_x1, outer_y1, outer_x2, outer_y2 = outer_rect
    inner_x1, inner_y1, inner_x2, inner_y2 = inner_rect
    return inner_x1 >= outer_x1 and inner_y1 >= outer_y1 and inner_x2 <= outer_x2 and inner_y2 <= outer_y2

def print_rectangle_contained(results, results2, image):
    flag = 0
    color1 = (0, 255, 0)  # Green color
    color2 = (255, 0, 0)
    thickness = 2
    reader = BarCodeReader()
    
    for box in zip(results[0].boxes.xyxy, results[0].boxes.conf):
        flag = 0
        for box2 in zip(results2[0].boxes.xywh, results2[0].boxes.conf):
            if is_rectangle_contained(np.array(box[0]), np.array(box2[0])):
                x1, y1, x2, y2 = np.array(box[0])
                rect1 = ((int(x1), int(y1)), (int(x2), int(y2)))
                cv2.rectangle(image, rect1[0], rect1[1], color1, thickness)
                
                x3, y3, width3, height3 = np.array(box2[0])
                x, y, width, height = int(x3 - (width3)/2), int(y3 - (height3)/2), int(width3), int(height3)
                region_img = image[y:y+height, x:x+width]
                region_img_gray = cv2.cvtColor(region_img, cv2.COLOR_BGR2GRAY)
                
                # Save the cropped region as a temporary file
                with tempfile.NamedTemporaryFile(suffix='.png', delete=False) as temp_file:
                    temp_file_path = temp_file.name
                    cv2.imwrite(temp_file_path, region_img_gray)
                
                barcode = reader.decode(temp_file_path)
                print('box:', np.array(box[0]), 'conf:', box[1], 'barcode:', np.array(box2[0]), 'conf:', box2[1], 'value:', barcode.raw)
                flag = 1
        
        if flag == 0:
            x1, y1, x2, y2 = np.array(box[0]) 
            rect2 = ((int(x1), int(y1)), (int(x2), int(y2)))
            cv2.rectangle(image, rect2[0], rect2[1], color2, thickness)
            print('box:', np.array(box[0]), 'does not contain barcode')
    
    cv2.imshow('image', image)

# Initialize the trained YOLO model
model_trained = YOLO('barcode_scanner.pt')

# Open the default camera (usually the built-in webcam)
cap = cv2.VideoCapture(0)

# Check if the camera opened successfully
if not cap.isOpened():
    print("Error: Unable to open camera.")
else:
    while True:
        # Read a frame from the camera
        ret, frame = cap.read()
        
        if not ret:
            print("Unable to read frame from camera.")
            break
        
        # Resize the frame if necessary
        # frame_resized = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)
        # image = frame_resized
        
        original_height, original_width, _ = frame.shape
        
        # Perform predictions using your YOLO models
        results = model_trained.predict(frame, imgsz=128)
        results2 = model_trained.predict(frame, imgsz=(original_height, original_width))
        
        # Process and display the results
        print_rectangle_contained(results, results2, frame)
        
        # Display the frame
        cv2.imshow('Live Camera', frame)
        
        # Check for 'q' key press to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Release the camera and close any open windows
cap.release()
cv2.destroyAllWindows()



0: 96x128 (no detections), 47.8ms
Speed: 2.0ms preprocess, 47.8ms inference, 1.1ms postprocess per image at shape (1, 3, 96, 128)

0: 480x640 4 class_0s, 291.1ms
Speed: 6.3ms preprocess, 291.1ms inference, 2.3ms postprocess per image at shape (1, 3, 480, 640)

0: 96x128 (no detections), 37.6ms
Speed: 0.8ms preprocess, 37.6ms inference, 1.0ms postprocess per image at shape (1, 3, 96, 128)

0: 480x640 4 class_0s, 177.1ms
Speed: 2.2ms preprocess, 177.1ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 96x128 (no detections), 23.1ms
Speed: 2.1ms preprocess, 23.1ms inference, 0.7ms postprocess per image at shape (1, 3, 96, 128)

0: 480x640 4 class_0s, 163.6ms
Speed: 2.5ms preprocess, 163.6ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

0: 96x128 (no detections), 23.5ms
Speed: 1.3ms preprocess, 23.5ms inference, 1.0ms postprocess per image at shape (1, 3, 96, 128)

0: 480x640 4 class_0s, 170.2ms
Speed: 2.7ms preprocess, 170.2ms inference, 1.5ms post

box: [      119.9      170.29      328.56      232.59] conf: tensor(0.3574) barcode: [     481.84      198.48       207.4      56.504] conf: tensor(0.9427) value: 
box: [      119.9      170.29      328.56      232.59] conf: tensor(0.3574) barcode: [     510.43      375.36      208.49      72.246] conf: tensor(0.9353) value: 

0: 96x128 2 class_0s, 21.4ms
Speed: 2.0ms preprocess, 21.4ms inference, 1.0ms postprocess per image at shape (1, 3, 96, 128)

0: 480x640 4 class_0s, 154.6ms
Speed: 2.7ms preprocess, 154.6ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)
box: [     75.123      351.81      301.77       429.8] conf: tensor(0.7471) barcode: [     518.65      384.61      242.71      85.957] conf: tensor(0.9599) value: 1234509876
box: [     75.123      351.81      301.77       429.8] conf: tensor(0.7471) barcode: [     188.65      392.67      224.97      77.573] conf: tensor(0.9582) value: 
box: [     396.13      340.81      639.49       428.3] conf: tensor(0.4500) ba

## image

In [11]:
import cv2
from zxing import BarCodeReader
import tempfile
from ultralytics import YOLO
import numpy as np


def is_rectangle_contained(outer_rect, inner_rect):
    outer_x1, outer_y1, outer_x2, outer_y2 = outer_rect
    inner_x1, inner_y1, inner_x2, inner_y2 = inner_rect
    return inner_x1 >= outer_x1 and inner_y1 >= outer_y1 and inner_x2 <= outer_x2 and inner_y2 <= outer_y2


def print_rectangle_contained(results, results2):
    image = results[0].plot() & results2[0].plot()
    flag = 0
    color1 = (0, 255, 0)  # Green color
    color2 = (255, 0, 0)
    thickness = 2
    reader = BarCodeReader()
    for box in zip(results[0].boxes.xyxy, results[0].boxes.conf):
        flag = 0
        for box2 in zip(results2[0].boxes.xywh, results2[0].boxes.conf):
            if is_rectangle_contained(np.array(box[0]), np.array(box2[0])):
                x1, y1, x2, y2 = np.array(box[0])
                rect1 = ((int(x1), int(y1)), (int(x2), int(y2)))
                cv2.rectangle(image, rect1[0], rect1[1], color1, thickness)
                x3, y3, width3, height3 = np.array(box2[0])
                x, y, width, height = int(x3 - (width3) / 2), int(y3 - (height3) / 2), int(width3), int(height3)
                region_img = image[y:y + height, x:x + width]
                region_img_gray = cv2.cvtColor(region_img, cv2.COLOR_BGR2GRAY)
                # Save the cropped region as a temporary file
                with tempfile.NamedTemporaryFile(suffix='.png', delete=False) as temp_file:
                    temp_file_path = temp_file.name
                    cv2.imwrite(temp_file_path, region_img_gray)

                # Decode barcode from the temporary file
                barcode = reader.decode(temp_file_path)

                print('box :', np.array(box[0]), 'conf:', box[1], 'barcode :', np.array(box2[0]), 'conf:', box2[1],
                      'value : ', barcode.raw)
                flag = 1

        if flag == 0:
            x1, y1, x2, y2 = np.array(box[0])
            rect2 = ((int(x1), int(y1)), (int(x2), int(y2)))
            cv2.rectangle(image, rect2[0], rect2[1], color2, thickness)
            print('box :', np.array(box[0]), 'does not contain barcode')

    cv2.imshow('Detected Barcodes', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


# Initialize the trained YOLO model
model_trained = YOLO('barcode_scanner.pt')

# Specify the path to the image file
image_path = 'Special_0131.jpg'

# Read the image
image = cv2.imread(image_path)

# Perform YOLO object detection on the image
original_height, original_width, _ = image.shape
results = model_trained.predict(image, imgsz=128)
results2 = model_trained.predict(image, imgsz=(original_height, original_width))

# Print rectangles containing barcodes and decode them
print_rectangle_contained(results, results2)



0: 96x128 2 class_0s, 43.4ms
Speed: 2.0ms preprocess, 43.4ms inference, 2.0ms postprocess per image at shape (1, 3, 96, 128)

WARNING ⚠️ imgsz=[720, 1280] must be multiple of max stride 32, updating to [736, 1280]
0: 736x1280 6 class_0s, 842.5ms
Speed: 15.7ms preprocess, 842.5ms inference, 3.0ms postprocess per image at shape (1, 3, 736, 1280)
box : [     356.55      131.76      761.98      236.09] conf: tensor(0.5574) barcode : [     547.99      187.73      409.76      96.312] conf: tensor(0.9643) value :  
box : [     356.55      131.76      761.98      236.09] conf: tensor(0.5574) barcode : [     776.19      327.35      431.51       62.44] conf: tensor(0.9510) value :  
box : [     356.55      131.76      761.98      236.09] conf: tensor(0.5574) barcode : [     465.66      619.27      640.69      109.96] conf: tensor(0.8097) value :  
box : [     356.55      131.76      761.98      236.09] conf: tensor(0.5574) barcode : [     649.51      602.14      531.49      99.835] conf: tensor

## trial

## open camera pyzbar

In [ ]:
import cv2
from ultralytics import YOLO
from pyzbar.pyzbar import decode
import matplotlib.pyplot as plt
barcode_scanner_model = YOLO('best.pt')


def decode_barcodes_from_regions(image, regions):
    decoded_barcodes = []
    annotated_image = image.copy()  # Create a copy of the original image for annotation
    for region in regions:
        x, y, width, height = region
        # Convert coordinates to integers
        x, y, width, height = int(x), int(y), int(width), int(height)
        # Crop the region from the image
        region_img = image[y:y+height, x:x+width]
        # Decode the barcode in the cropped region
        detected_barcodes = decode(region_img)
        decoded_barcodes.extend(detected_barcodes)
        # Draw rectangle around the region
        cv2.rectangle(annotated_image, (x, y), (x + width, y + height), (0, 255, 0), 2)
    return decoded_barcodes, annotated_image

def get_box_dimensions(results):
    box_dimensions = []
    for box, box2 in zip(results[0].boxes.xyxy, results[0].boxes.xywh):
        box_dimensions.append([box[0].item(), box[1].item(), box2[2].item(), box2[3].item()])
    return box_dimensions

# Load the image
image_path = 'special_0131.jpg'
image = cv2.imread(image_path)

# Load the results from barcode scanner model (assuming you have defined `barcode_scanner_model`)
results = barcode_scanner_model.predict(image_path, imgsz=(image.shape[1], image.shape[0]))

# Get the box dimensions from the results
box_dimensions = get_box_dimensions(results)

# Initialize the regions to search for barcodes
regions_to_search = []

# Fill the regions with box dimensions from results
for box_dim in box_dimensions:
    x, y, width, height = box_dim
    regions_to_search.append((x, y, width, height))

# Decode barcodes from the specified regions and get annotated image
decoded_barcodes, annotated_image = decode_barcodes_from_regions(image, regions_to_search)

# Print the decoded barcodes
if decoded_barcodes:
    for barcode in decoded_barcodes:
        print(f"Found barcode:"
              f"\n Data:    {barcode.data}"
              f"\n Type:    {barcode.type}")
else:
    print("No barcodes found in the specified regions.")

# Display the annotated image
plt.imshow(cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB))
plt.title('Annotated Image with Detected Barcodes')
plt.axis('off')
plt.show()



WARNING ⚠️ imgsz=[1280, 720] must be multiple of max stride 32, updating to [1280, 736]
image 1/1 C:\Users\Madhur.Gauri\Desktop\COMPUTER _VISION_WORK_\Read_barcode_code_july\special_0131.jpg: 416x736 (no detections), 355.4ms
Speed: 11.1ms preprocess, 355.4ms inference, 2.0ms postprocess per image at shape (1, 3, 416, 736)
No barcodes found in the specified regions.


## open camera zxing

In [1]:
import cv2
from zxing import BarCodeReader
import tempfile
import numpy as np
from ultralytics import YOLO

def is_rectangle_contained(outer_rect, inner_rect):
    outer_x1, outer_y1, outer_x2, outer_y2 = outer_rect
    inner_x1, inner_y1, inner_x2, inner_y2 = inner_rect
    return inner_x1 >= outer_x1 and inner_y1 >= outer_y1 and inner_x2 <= outer_x2 and inner_y2 <= outer_y2

def print_rectangle_contained(results, results2, image):
    flag = 0
    color1 = (0, 255, 0)  # Green color
    color2 = (255, 0, 0)
    thickness = 2
    reader = BarCodeReader()
    
    for box in zip(results[0].boxes.xyxy, results[0].boxes.conf):
        flag = 0
        for box2 in zip(results2[0].boxes.xywh, results2[0].boxes.conf):
            if is_rectangle_contained(np.array(box[0]), np.array(box2[0])):
                x1, y1, x2, y2 = np.array(box[0])
                rect1 = ((int(x1), int(y1)), (int(x2), int(y2)))
                cv2.rectangle(image, rect1[0], rect1[1], color1, thickness)
                
                x3, y3, width3, height3 = np.array(box2[0])
                x, y, width, height = int(x3 - (width3)/2), int(y3 - (height3)/2), int(width3), int(height3)
                region_img = image[y:y+height, x:x+width]
                region_img_gray = cv2.cvtColor(region_img, cv2.COLOR_BGR2GRAY)
                
                # Save the cropped region as a temporary file
                with tempfile.NamedTemporaryFile(suffix='.png', delete=False) as temp_file:
                    temp_file_path = temp_file.name
                    cv2.imwrite(temp_file_path, region_img_gray)
                
                barcode = reader.decode(temp_file_path)
                print('box:', np.array(box[0]), 'conf:', box[1], 'barcode:', np.array(box2[0]), 'conf:', box2[1], 'value:', barcode.raw)
                flag = 1
        
        if flag == 0:
            x1, y1, x2, y2 = np.array(box[0]) 
            rect2 = ((int(x1), int(y1)), (int(x2), int(y2)))
            cv2.rectangle(image, rect2[0], rect2[1], color2, thickness)
            print('box:', np.array(box[0]), 'does not contain barcode')
    
    cv2.imshow('image', image)

# Initialize the trained YOLO model
model_trained = YOLO('barcode_scanner.pt')

# Open the default camera (usually the built-in webcam)
cap = cv2.VideoCapture(0)

# Check if the camera opened successfully
if not cap.isOpened():
    print("Error: Unable to open camera.")
else:
    while True:
        # Read a frame from the camera
        ret, frame = cap.read()
        
        if not ret:
            print("Unable to read frame from camera.")
            break
        
        # Resize the frame if necessary
        # frame_resized = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)
        # image = frame_resized
        
        original_height, original_width, _ = frame.shape
        
        # Perform predictions using your YOLO models
        results = model_trained.predict(frame, imgsz=128)
        results2 = model_trained.predict(frame, imgsz=(original_height, original_width))
        
        # Process and display the results
        print_rectangle_contained(results, results2, frame)
        
        # Display the frame
        cv2.imshow('Live Camera', frame)
        
        # Check for 'q' key press to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Release the camera and close any open windows
cap.release()
cv2.destroyAllWindows()



0: 96x128 (no detections), 53.1ms
Speed: 2.0ms preprocess, 53.1ms inference, 1.8ms postprocess per image at shape (1, 3, 96, 128)

0: 480x640 2 class_0s, 338.2ms
Speed: 8.2ms preprocess, 338.2ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 96x128 (no detections), 27.2ms
Speed: 0.0ms preprocess, 27.2ms inference, 0.0ms postprocess per image at shape (1, 3, 96, 128)

0: 480x640 2 class_0s, 160.5ms
Speed: 6.2ms preprocess, 160.5ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 96x128 (no detections), 27.7ms
Speed: 2.0ms preprocess, 27.7ms inference, 0.9ms postprocess per image at shape (1, 3, 96, 128)

0: 480x640 2 class_0s, 160.3ms
Speed: 2.0ms preprocess, 160.3ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 96x128 (no detections), 27.9ms
Speed: 1.5ms preprocess, 27.9ms inference, 0.0ms postprocess per image at shape (1, 3, 96, 128)

0: 480x640 2 class_0s, 151.5ms
Speed: 1.6ms preprocess, 151.5ms inference, 1.0ms post


0: 480x640 4 class_0s, 273.6ms
Speed: 5.2ms preprocess, 273.6ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 96x128 (no detections), 40.2ms
Speed: 1.5ms preprocess, 40.2ms inference, 1.0ms postprocess per image at shape (1, 3, 96, 128)

0: 480x640 4 class_0s, 285.4ms
Speed: 3.4ms preprocess, 285.4ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 96x128 (no detections), 50.0ms
Speed: 3.0ms preprocess, 50.0ms inference, 1.5ms postprocess per image at shape (1, 3, 96, 128)

0: 480x640 4 class_0s, 277.7ms
Speed: 4.5ms preprocess, 277.7ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 96x128 (no detections), 40.2ms
Speed: 1.7ms preprocess, 40.2ms inference, 1.0ms postprocess per image at shape (1, 3, 96, 128)

0: 480x640 4 class_0s, 286.9ms
Speed: 5.0ms preprocess, 286.9ms inference, 2.7ms postprocess per image at shape (1, 3, 480, 640)

0: 96x128 (no detections), 40.8ms
Speed: 1.5ms preprocess, 40.8ms inference, 1.5ms pos


0: 480x640 4 class_0s, 148.7ms
Speed: 5.5ms preprocess, 148.7ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 96x128 (no detections), 27.2ms
Speed: 0.5ms preprocess, 27.2ms inference, 1.0ms postprocess per image at shape (1, 3, 96, 128)

0: 480x640 4 class_0s, 149.1ms
Speed: 2.1ms preprocess, 149.1ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 96x128 (no detections), 20.8ms
Speed: 2.5ms preprocess, 20.8ms inference, 0.0ms postprocess per image at shape (1, 3, 96, 128)

0: 480x640 4 class_0s, 140.6ms
Speed: 2.8ms preprocess, 140.6ms inference, 3.2ms postprocess per image at shape (1, 3, 480, 640)

0: 96x128 (no detections), 39.7ms
Speed: 1.4ms preprocess, 39.7ms inference, 1.0ms postprocess per image at shape (1, 3, 96, 128)

0: 480x640 4 class_0s, 151.5ms
Speed: 1.9ms preprocess, 151.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 96x128 (no detections), 32.6ms
Speed: 0.5ms preprocess, 32.6ms inference, 1.0ms pos

box: [     376.05      367.13      630.85      450.12] conf: tensor(0.3461) barcode: [     487.99      406.76      236.25      77.872] conf: tensor(0.9449) value: 

0: 96x128 (no detections), 23.9ms
Speed: 1.0ms preprocess, 23.9ms inference, 0.0ms postprocess per image at shape (1, 3, 96, 128)

0: 480x640 4 class_0s, 143.7ms
Speed: 3.9ms preprocess, 143.7ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 96x128 (no detections), 20.9ms
Speed: 1.4ms preprocess, 20.9ms inference, 1.0ms postprocess per image at shape (1, 3, 96, 128)

0: 480x640 4 class_0s, 150.5ms
Speed: 2.0ms preprocess, 150.5ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 640)

0: 96x128 (no detections), 26.6ms
Speed: 2.0ms preprocess, 26.6ms inference, 1.0ms postprocess per image at shape (1, 3, 96, 128)

0: 480x640 3 class_0s, 145.0ms
Speed: 2.0ms preprocess, 145.0ms inference, 1.6ms postprocess per image at shape (1, 3, 480, 640)

0: 96x128 (no detections), 23.9ms
Speed: 1.6ms prepr

## open camera pyzbar different code

In [2]:
import cv2
from pyzbar.pyzbar import decode

def decode_barcodes(image_path):
  """Decodes barcodes from an image file.

  Args:
    image_path: Path to the image file.

  Returns:
    A list of tuples, where each tuple contains the barcode data and type.
  """

  img = cv2.imread(image_path)
  barcodes = decode(img)

  barcode_info = []
  for barcode in barcodes:
    barcode_data = barcode.data.decode('utf-8')
    barcode_type = barcode.type
    barcode_info.append((barcode_data, barcode_type))

  return barcode_info

# Example usage:
image_path = 'Special_0131.jpg'
barcode_data = decode_barcodes(image_path)
print(barcode_data)


[('104000000000388768', 'CODE128'), ('3837404503', 'CODE128'), ('383740450301', 'CODE128'), ('401', 'CODE128')]


In [1]:
import cv2
from pyzbar.pyzbar import decode

def decode_barcodes(image_path):
  img = cv2.imread(image_path)

  # Preprocessing (optional):
  # img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
  # _, img = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)  # Apply thresholding

  barcodes = decode(img)

  barcode_data = []
  for barcode in barcodes:
    barcode_info = {
      'data': barcode.data.decode('utf-8'),
      'type': barcode.type
    }
    barcode_data.append(barcode_info)

  return barcode_data

# Example usage:
image_path = 'Special_0131.jpg'
barcode_data = decode_barcodes(image_path)
print(barcode_data)


[{'data': '104000000000388768', 'type': 'CODE128'}, {'data': '3837404503', 'type': 'CODE128'}, {'data': '383740450301', 'type': 'CODE128'}, {'data': '401', 'type': 'CODE128'}]


In [ ]:
import cv2
from pyzbar.pyzbar import decode

def decode_barcodes_from_camera():
    cap = cv2.VideoCapture(0)  # Access the default camera

    while True:
        ret, frame = cap.read()

        if not ret:
            break

        barcodes = decode(frame)

        for barcode in barcodes:
            barcode_data = barcode.data.decode('utf-8')
            barcode_type = barcode.type
            print(f"Barcode Data: {barcode_data}, Type: {barcode_type}")

        cv2.imshow('Barcode Scanner', frame)

        if cv2.waitKey(1) & 0xFF == 27:  # Press 'Esc' to exit
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    decode_barcodes_from_camera()


In [2]:
!pip install zxing


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



     ---------------------------------------- 0.0/689.6 kB ? eta -:--:--
     ---- ---------------------------------- 71.7/689.6 kB 2.0 MB/s eta 0:00:01
     ------ ------------------------------- 112.6/689.6 kB 1.3 MB/s eta 0:00:01
     -------- ----------------------------- 153.6/689.6 kB 1.1 MB/s eta 0:00:01
     ---------- --------------------------- 194.6/689.6 kB 1.2 MB/s eta 0:00:01
     --------------- ---------------------- 276.5/689.6 kB 1.2 MB/s eta 0:00:01
     ------------------ ------------------- 337.9/689.6 kB 1.3 MB/s eta 0:00:01
     ----------------------- -------------- 430.1/689.6 kB 1.4 MB/s eta 0:00:01
     ---------------------------- --------- 522.2/689.6 kB 1.5 MB/s eta 0:00:01
     -------------------------------------  686.1/689.6 kB 1.7 MB/s eta 0:00:01
     -------------------------------------- 689.6/689.6 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for zxin

In [ ]:
import cv2
from zxing import BarCodeReader
import tempfile
import numpy as np
from ultralytics import YOLO

def is_rectangle_contained(outer_rect, inner_rect):
    outer_x1, outer_y1, outer_x2, outer_y2 = outer_rect
    inner_x1, inner_y1, inner_x2, inner_y2 = inner_rect
    return inner_x1 >= outer_x1 and inner_y1 >= outer_y1 and inner_x2 <= outer_x2 and inner_y2 <= outer_y2

def print_rectangle_contained(results, results2, image):
    flag = 0
    color1 = (0, 255, 0)  # Green color
    color2 = (255, 0, 0)
    thickness = 2
    reader = BarCodeReader()
    
    for box in zip(results[0].boxes.xyxy, results[0].boxes.conf):
        flag = 0
        for box2 in zip(results2[0].boxes.xywh, results2[0].boxes.conf):
            if is_rectangle_contained(np.array(box[0]), np.array(box2[0])):
                x1, y1, x2, y2 = np.array(box[0])
                rect1 = ((int(x1), int(y1)), (int(x2), int(y2)))
                cv2.rectangle(image, rect1[0], rect1[1], color1, thickness)
                
                x3, y3, width3, height3 = np.array(box2[0])
                x, y, width, height = int(x3 - (width3)/2), int(y3 - (height3)/2), int(width3), int(height3)
                region_img = image[y:y+height, x:x+width]
                region_img_gray = cv2.cvtColor(region_img, cv2.COLOR_BGR2GRAY)
                
                # Save the cropped region as a temporary file
                with tempfile.NamedTemporaryFile(suffix='.png', delete=False) as temp_file:
                    temp_file_path = temp_file.name
                    cv2.imwrite(temp_file_path, region_img_gray)
                
                barcode = reader.decode(temp_file_path)
                print('box:', np.array(box[0]), 'conf:', box[1], 'barcode:', np.array(box2[0]), 'conf:', box2[1], 'value:', barcode.raw)
                flag = 1
        
        if flag == 0:
            x1, y1, x2, y2 = np.array(box[0]) 
            rect2 = ((int(x1), int(y1)), (int(x2), int(y2)))
            cv2.rectangle(image, rect2[0], rect2[1], color2, thickness)
            print('box:', np.array(box[0]), 'does not contain barcode')
    
    cv2.imshow('image', image)

# Initialize the trained YOLO model
model_trained = YOLO('barcode_scanner.pt')

# Open the default camera (usually the built-in webcam)
cap = cv2.VideoCapture(0)

# Check if the camera opened successfully
if not cap.isOpened():
    print("Error: Unable to open camera.")
else:
    while True:
        # Read a frame from the camera
        ret, frame = cap.read()
        
        if not ret:
            print("Unable to read frame from camera.")
            break
        
        # Resize the frame if necessary
        # frame_resized = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)
        # image = frame_resized
        
        original_height, original_width, _ = frame.shape
        
        # Perform predictions using your YOLO models
        results = model_trained.predict(frame, imgsz=128)
        results2 = model_trained.predict(frame, imgsz=(original_height, original_width))
        
        # Process and display the results
        print_rectangle_contained(results, results2, frame)
        
        # Display the frame
        cv2.imshow('Live Camera', frame)
        
        # Check for 'q' key press to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Release the camera and close any open windows
cap.release()
cv2.destroyAllWindows()



0: 96x128 (no detections), 126.9ms
Speed: 8.1ms preprocess, 126.9ms inference, 6.8ms postprocess per image at shape (1, 3, 96, 128)

0: 480x640 4 class_0s, 515.7ms
Speed: 5.7ms preprocess, 515.7ms inference, 11.8ms postprocess per image at shape (1, 3, 480, 640)

0: 96x128 (no detections), 43.6ms
Speed: 3.6ms preprocess, 43.6ms inference, 1.5ms postprocess per image at shape (1, 3, 96, 128)

0: 480x640 4 class_0s, 232.0ms
Speed: 11.6ms preprocess, 232.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 96x128 (no detections), 44.2ms
Speed: 6.4ms preprocess, 44.2ms inference, 1.0ms postprocess per image at shape (1, 3, 96, 128)

0: 480x640 4 class_0s, 240.2ms
Speed: 3.7ms preprocess, 240.2ms inference, 25.1ms postprocess per image at shape (1, 3, 480, 640)

0: 96x128 (no detections), 45.3ms
Speed: 2.6ms preprocess, 45.3ms inference, 1.0ms postprocess per image at shape (1, 3, 96, 128)

0: 480x640 4 class_0s, 242.5ms
Speed: 5.4ms preprocess, 242.5ms inference, 1.1ms

box: [     165.33      314.87      584.06      387.22] conf: tensor(0.3347) barcode: [     215.58      345.69      87.463      59.046] conf: tensor(0.9512) value: 
box: [     165.33      314.87      584.06      387.22] conf: tensor(0.3347) barcode: [     439.62      353.01      279.06      69.865] conf: tensor(0.9491) value: 
box: [     165.33      314.87      584.06      387.22] conf: tensor(0.3347) barcode: [     246.16      450.27      181.31      59.459] conf: tensor(0.9305) value: 



0: 96x128 1 class_0, 86.3ms
Speed: 6.5ms preprocess, 86.3ms inference, 3.3ms postprocess per image at shape (1, 3, 96, 128)



box: [     165.33      314.87      584.06      387.22] conf: tensor(0.3347) barcode: [     476.65      458.47      210.07      43.051] conf: tensor(0.9027) value: 


0: 480x640 2 class_0s, 461.5ms
Speed: 7.7ms preprocess, 461.5ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)


...

0: 480x640 2 class_0s, 576.7ms
Speed: 6.9ms preprocess, 576.7ms inference, 5.0ms postprocess per image at shape (1, 3, 480, 640)


box: [     165.82      364.48         640      455.11] conf: tensor(0.7014) barcode: [     488.11      410.54      303.79      78.844] conf: tensor(0.9548) value: 



0: 96x128 (no detections), 53.7ms
Speed: 4.0ms preprocess, 53.7ms inference, 2.6ms postprocess per image at shape (1, 3, 96, 128)



box: [     165.82      364.48         640      455.11] conf: tensor(0.7014) barcode: [      226.6      404.18      118.33      73.179] conf: tensor(0.9524) value: 


0: 480x640 (no detections), 499.7ms
Speed: 9.9ms preprocess, 499.7ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 96x128 (no detections), 70.4ms
Speed: 2.0ms preprocess, 70.4ms inference, 1.1ms postprocess per image at shape (1, 3, 96, 128)

0: 480x640 (no detections), 395.5ms
Speed: 6.6ms preprocess, 395.5ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 96x128 (no detections), 74.1ms
Speed: 2.0ms preprocess, 74.1ms inference, 1.5ms postprocess per image at shape (1, 3, 96, 128)

0: 480x640 (no detections), 424.9ms
Speed: 14.4ms preprocess, 424.9ms inference, 2.2ms postprocess per image at shape (1, 3, 480, 640)

0: 96x128 (no detections), 55.8ms
Speed: 2.1ms preprocess, 55.8ms inference, 1.0ms postprocess per image at shape (1, 3, 96, 128)

0: 480x640 (no detections), 382.4ms
Speed: 8.2ms preprocess, 382.4ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)

0: 96x128 (no detections), 53.6ms
Speed: 1.7ms preprocess, 53.6ms 

In [ ]:
## set

In [7]:
import cv2
from zxing import BarCodeReader
import tempfile
import numpy as np
from ultralytics import YOLO

def is_rectangle_contained(outer_rect, inner_rect):
    outer_x1, outer_y1, outer_x2, outer_y2 = outer_rect
    inner_x1, inner_y1, inner_x2, inner_y2 = inner_rect
    return inner_x1 >= outer_x1 and inner_y1 >= outer_y1 and inner_x2 <= outer_x2 and inner_y2 <= outer_y2

def print_rectangle_contained(results, results2, decoded_barcodes):
    image = results[0].plot() & results2[0].plot()
    flag = 0
    color1 = (0, 255, 0)  # Green color
    color2 = (255, 0, 0)  # Red color
    thickness = 2
    reader = BarCodeReader()
    
    for box in zip(results[0].boxes.xyxy, results[0].boxes.conf):
        flag = 0
        for box2 in zip(results2[0].boxes.xywh, results2[0].boxes.conf):
            if is_rectangle_contained(np.array(box[0]), np.array(box2[0])):
                x1, y1, x2, y2 = np.array(box[0])
                rect1 = ((int(x1), int(y1)), (int(x2), int(y2)))
                cv2.rectangle(image, rect1[0], rect1[1], color1, thickness)
                
                x3, y3, width3, height3 = np.array(box2[0])
                x, y, width, height = int(x3 - (width3) / 2), int(y3 - (height3) / 2), int(width3), int(height3)
                region_img = image[y:y + height, x:x + width]
                region_img_gray = cv2.cvtColor(region_img, cv2.COLOR_BGR2GRAY)
                
                # Save the cropped region as a temporary file
                with tempfile.NamedTemporaryFile(suffix='.png', delete=False) as temp_file:
                    temp_file_path = temp_file.name
                    cv2.imwrite(temp_file_path, region_img_gray)
                    
                barcode = reader.decode(temp_file_path)
                
                if barcode and barcode.raw not in decoded_barcodes:
                    decoded_barcodes.add(barcode.raw)
                    print('box :', np.array(box[0]), 'conf:', box[1], 'barcode :', np.array(box2[0]), 'conf:', box2[1], 'value :', barcode.raw)
                    flag = 1
                else:
                    # Clean up the temporary file
                    os.remove(temp_file_path)
            
        if flag == 0:
            x1, y1, x2, y2 = np.array(box[0])
            rect2 = ((int(x1), int(y1)), (int(x2), int(y2)))
            cv2.rectangle(image, rect2[0], rect2[1], color2, thickness)
            print('box :', np.array(box[0]), 'does not contain barcode')

    cv2.imshow('image', image)

# Initialize the trained YOLO model
model_trained = YOLO('barcode_scanner.pt')

# Specify the path to the video file
video_path = 'video.mp4'

# Open the video file for reading
cap = cv2.VideoCapture(video_path)
frame_count = 0
decoded_barcodes = set()  # Set to keep track of decoded barcodes

# Check if the video opened successfully
if not cap.isOpened():
    print("Error: Unable to open video.")
else:
    # Loop to read and process each frame
    while True:
        # Read a frame from the video
        ret, frame = cap.read()
        frame_count += 1
        
        # Check if the frame was read successfully
        if not ret:
            print("End of video or unable to read frame.")
            break
        
        if frame_count % 20 != 0:
            continue
        
        image = frame
        original_height, original_width, _ = image.shape
        
        # Use trained models for prediction
        model_trained = YOLO('best_1.pt')
        barcode_scanner_model = YOLO('barcode_scanner.pt')
        
        results = model_trained.predict(frame, imgsz=128)
        results2 = barcode_scanner_model.predict(frame, imgsz=(original_height, original_width))
        print_rectangle_contained(results, results2, decoded_barcodes)
        
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


FileNotFoundError: [Errno 2] No such file or directory: 'best_1.pt'

In [9]:
import os

In [10]:
import cv2
from zxing import BarCodeReader
import tempfile
import numpy as np
from ultralytics import YOLO

def is_rectangle_contained(outer_rect, inner_rect):
    outer_x1, outer_y1, outer_x2, outer_y2 = outer_rect
    inner_x1, inner_y1, inner_x2, inner_y2 = inner_rect
    return inner_x1 >= outer_x1 and inner_y1 >= outer_y1 and inner_x2 <= outer_x2 and inner_y2 <= outer_y2

def print_rectangle_contained(results, results2, image):
    flag = 0
    color1 = (0, 255, 0)  # Green color
    color2 = (255, 0, 0)
    thickness = 2
    reader = BarCodeReader()
    
    for box in zip(results[0].boxes.xyxy, results[0].boxes.conf):
        flag = 0
        for box2 in zip(results2[0].boxes.xywh, results2[0].boxes.conf):
            if is_rectangle_contained(np.array(box[0]), np.array(box2[0])):
                x1, y1, x2, y2 = np.array(box[0])
                rect1 = ((int(x1), int(y1)), (int(x2), int(y2)))
                cv2.rectangle(image, rect1[0], rect1[1], color1, thickness)
                
                x3, y3, width3, height3 = np.array(box2[0])
                x, y, width, height = int(x3 - (width3)/2), int(y3 - (height3)/2), int(width3), int(height3)
                region_img = image[y:y+height, x:x+width]
                region_img_gray = cv2.cvtColor(region_img, cv2.COLOR_BGR2GRAY)
                
                # Save the cropped region as a temporary file
                with tempfile.NamedTemporaryFile(suffix='.png', delete=False) as temp_file:
                    temp_file_path = temp_file.name
                    cv2.imwrite(temp_file_path, region_img_gray)
                
                barcode = reader.decode(temp_file_path)
                
                if barcode and barcode.raw not in decoded_barcodes:
                    decoded_barcodes.add(barcode.raw)
                    print('box :', np.array(box[0]), 'conf:', box[1], 'barcode :', np.array(box2[0]), 'conf:', box2[1], 'value :', barcode.raw)
                    flag = 1
                else:
                    # Clean up the temporary file
                    os.remove(temp_file_path)
        
        if flag == 0:
            x1, y1, x2, y2 = np.array(box[0]) 
            rect2 = ((int(x1), int(y1)), (int(x2), int(y2)))
            cv2.rectangle(image, rect2[0], rect2[1], color2, thickness)
            print('box:', np.array(box[0]), 'does not contain barcode')
    
    cv2.imshow('image', image)

# Initialize the trained YOLO model
model_trained = YOLO('barcode_scanner.pt')

# Open the default camera (usually the built-in webcam)
cap = cv2.VideoCapture(0)

# Check if the camera opened successfully
if not cap.isOpened():
    print("Error: Unable to open camera.")
else:
    while True:
        # Read a frame from the camera
        ret, frame = cap.read()
        
        if not ret:
            print("Unable to read frame from camera.")
            break
        
        # Resize the frame if necessary
        # frame_resized = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)
        # image = frame_resized
        
        original_height, original_width, _ = frame.shape
        
        # Perform predictions using your YOLO models
        results = model_trained.predict(frame, imgsz=128)
        results2 = model_trained.predict(frame, imgsz=(original_height, original_width))
        
        # Process and display the results
        print_rectangle_contained(results, results2, frame)
        
        # Display the frame
        cv2.imshow('Live Camera', frame)
        
        # Check for 'q' key press to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Release the camera and close any open windows
cap.release()
cv2.destroyAllWindows()



0: 96x128 1 class_0, 21.0ms
Speed: 0.0ms preprocess, 21.0ms inference, 0.5ms postprocess per image at shape (1, 3, 96, 128)

0: 480x640 4 class_0s, 107.8ms
Speed: 3.5ms preprocess, 107.8ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)
box: [     373.01      47.527      599.22      124.82] does not contain barcode

0: 96x128 (no detections), 21.1ms
Speed: 0.5ms preprocess, 21.1ms inference, 1.1ms postprocess per image at shape (1, 3, 96, 128)

0: 480x640 4 class_0s, 98.4ms
Speed: 2.6ms preprocess, 98.4ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 96x128 (no detections), 14.1ms
Speed: 0.8ms preprocess, 14.1ms inference, 1.2ms postprocess per image at shape (1, 3, 96, 128)

0: 480x640 4 class_0s, 92.6ms
Speed: 2.8ms preprocess, 92.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 96x128 (no detections), 16.7ms
Speed: 0.7ms preprocess, 16.7ms inference, 0.0ms postprocess per image at shape (1, 3, 96, 128)

0: 480x640 4 cl

In [12]:
import cv2
from pyzbar.pyzbar import decode

def decode_barcodes(frame):
    # Decode barcodes in the frame
    barcodes = decode(frame)

    barcode_data = []
    for barcode in barcodes:
        # Extract data and type from barcode
        barcode_info = {
            'data': barcode.data.decode('utf-8'),
            'type': barcode.type
        }
        barcode_data.append(barcode_info)
        
        # Draw rectangle around detected barcode
        (x, y, w, h) = barcode.rect
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        
        # Display barcode data on the frame
        barcode_text = f"{barcode_info['data']} ({barcode_info['type']})"
        cv2.putText(frame, barcode_text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    return barcode_data

def main():
    # Open a connection to the camera (0 is typically the default camera)
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("Error: Unable to access the camera.")
        return

    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()
        if not ret:
            print("Error: Unable to read frame.")
            break

        # Decode barcodes in the frame
        barcode_data = decode_barcodes(frame)
        
        # Print decoded barcodes data
        for data in barcode_data:
            print(f"Decoded barcode data: {data['data']} (Type: {data['type']})")
        
        # Display the resulting frame
        cv2.imshow('Barcode Scanner', frame)
        
        # Exit the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the camera and close windows
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


Decoded barcode data: 1234509876 (Type: CODE128)
Decoded barcode data: 1234509876 (Type: CODE128)
Decoded barcode data: 1234509876 (Type: CODE128)
Decoded barcode data: 1234509876 (Type: CODE128)
Decoded barcode data: 1234509876 (Type: CODE128)
Decoded barcode data: 1234509876 (Type: CODE128)
Decoded barcode data: 1234509876 (Type: CODE128)
Decoded barcode data: 1234509876 (Type: CODE128)
Decoded barcode data: Hii Sir (Type: CODE128)
Decoded barcode data: 1234509876 (Type: CODE128)
Decoded barcode data: Hii Sir (Type: CODE128)
Decoded barcode data: 1234509876 (Type: CODE128)
Decoded barcode data: 1234509876 (Type: CODE128)
Decoded barcode data: 1234509876 (Type: CODE128)
Decoded barcode data: Hii Sir (Type: CODE128)
Decoded barcode data: 1234509876 (Type: CODE128)
Decoded barcode data: 1234509876 (Type: CODE128)
Decoded barcode data: 1234509876 (Type: CODE128)
Decoded barcode data: 1234509876 (Type: CODE128)
Decoded barcode data: 1234509876 (Type: CODE128)
Decoded barcode data: 1234509

In [19]:
import cv2
from pyzbar.pyzbar import decode

def decode_barcodes(frame, processed_barcodes):
    # Decode barcodes in the frame
    barcodes = decode(frame)

    barcode_data = []
    for barcode in barcodes:
        # Extract data and type from barcode
        barcode_value = barcode.data.decode('utf-8')
        barcode_info = {
            'data': barcode_value,
            'type': barcode.type
        }
        
        # Check if barcode is already processed
        if barcode_value not in processed_barcodes:
            processed_barcodes.add(barcode_value)
            
            # Draw rectangle around detected barcode
            (x, y, w, h) = barcode.rect
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            
            # Display barcode data on the frame
            barcode_text = f"{barcode_info['data']} ({barcode_info['type']})"
            cv2.putText(frame, barcode_text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            
            # Add barcode data to the list for printing
            barcode_data.append(barcode_info)
    
    return barcode_data

def main():
    # Open a connection to the camera (0 is typically the default camera)
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("Error: Unable to access the camera.")
        return
    
    # Set to keep track of processed barcode values
    processed_barcodes = set()

    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()
        if not ret:
            print("Error: Unable to read frame.")
            break
        
        # Decode barcodes in the frame
        barcode_data = decode_barcodes(frame, processed_barcodes)
        
        # Print decoded barcodes data
        for data in barcode_data:
            print(f"Decoded barcode data: {data['data']} (Type: {data['type']})")
        
        # Display the resulting frame
        cv2.imshow('Barcode Scanner', frame)
        
        # Exit the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the camera and close windows
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


Decoded barcode data: Hii Sir (Type: CODE128)
Decoded barcode data: ABCD (Type: CODE128)
Decoded barcode data: IJKL (Type: CODE128)
Decoded barcode data: EFGH (Type: CODE128)
Decoded barcode data: MNOP1234 (Type: CODE128)


In [ ]:
import cv2
from pyzbar.pyzbar import decode

def decode_barcodes(frame, processed_barcodes):
    # Decode barcodes in the frame
    barcodes = decode(frame)

    barcode_data = []
    for barcode in barcodes:
        # Extract data and type from barcode
        barcode_value = barcode.data.decode('utf-8')
        barcode_info = {
            'data': barcode_value,
            'type': barcode.type
        }
        
        # Check if barcode is already processed
        if barcode_value not in processed_barcodes:
            processed_barcodes.add(barcode_value)
            
            # Draw rectangle around detected barcode
            (x, y, w, h) = barcode.rect
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            
            # Display barcode data on the frame
            barcode_text = f"{barcode_info['data']} ({barcode_info['type']})"
            cv2.putText(frame, barcode_text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            
            # Add barcode data to the list for printing
            barcode_data.append(barcode_info)
    
    return barcode_data

def main():
    # Open a connection to the camera (0 is typically the default camera)
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("Error: Unable to access the camera.")
        return
    
    # Set to keep track of processed barcode values
    processed_barcodes = set()

    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()
        if not ret:
            print("Error: Unable to read frame.")
            break
        
        # Decode barcodes in the frame
        barcode_data = decode_barcodes(frame, processed_barcodes)
        
        # Print decoded barcodes data
        for data in barcode_data:
            print(f"Decoded barcode data: {data['data']} (Type: {data['type']})")
        
        # Display the resulting frame
        cv2.imshow('Barcode Scanner', frame)
        
        # Exit the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the camera and close windows
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


In [3]:
import cv2
from pyzbar.pyzbar import decode

def decode_barcodes(frame, processed_barcodes):
    # Decode barcodes in the frame
    barcodes = decode(frame)

    barcode_data = []
    for barcode in barcodes:
        # Extract data and type from barcode
        barcode_value = barcode.data.decode('utf-8')
        barcode_info = {
            'data': barcode_value,
            'type': barcode.type
        }
        
        # Check if barcode is already processed
        if barcode_value not in processed_barcodes:
            processed_barcodes.add(barcode_value)
            
            # Draw rectangle around detected barcode
            (x, y, w, h) = barcode.rect
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            
            # Display barcode data on the frame
            barcode_text = f"{barcode_info['data']} ({barcode_info['type']})"
            cv2.putText(frame, barcode_text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            
            # Add barcode data to the list for printing
            barcode_data.append(barcode_info)
    
    return barcode_data

def main():
    # Path to the video file
    video_path = 'video_4.mp4'

    # Open a connection to the video file
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error: Unable to open the video file.")
        return
    
    # Set to keep track of processed barcode values
    processed_barcodes = set()

    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()
        if not ret:
            print("Video stream ended or error occurred.")
            break
        
        # Decode barcodes in the frame
        barcode_data = decode_barcodes(frame, processed_barcodes)
        
        # Print decoded barcodes data
        for data in barcode_data:
            print(f"Decoded barcode data: {data['data']} (Type: {data['type']})")
        
        # Display the resulting frame
        cv2.imshow('Barcode Scanner', frame)
        
        # Exit the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the video capture and close windows
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


Decoded barcode data: 849520 (Type: I25)
Video stream ended or error occurred.


In [ ]:
haan